In [5]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1872, -0.1244,  0.1466,  0.1753, -0.0147, -0.1442, -0.0248,  0.0195,
         -0.1090, -0.1205],
        [-0.0954, -0.2453,  0.1851,  0.1718, -0.0539, -0.2331, -0.0423,  0.0652,
         -0.1793, -0.1164]], grad_fn=<AddmmBackward0>)

In [6]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[ 0.0470,  0.1079,  0.0735, -0.0724, -0.1907,  0.2286,  0.2757,  0.2015,
         -0.0142,  0.0210],
        [ 0.0261,  0.0706, -0.1461,  0.0669, -0.1083,  0.0150,  0.3124,  0.1079,
         -0.0213, -0.1054]], grad_fn=<AddmmBackward0>)

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0530, -0.2226,  0.1594, -0.1091,  0.1478,  0.1370, -0.1917, -0.0789,
         -0.3660,  0.2394],
        [-0.1251, -0.1870,  0.0942, -0.0669,  0.1056,  0.2753, -0.1908, -0.0080,
         -0.3408,  0.1980]], grad_fn=<AddmmBackward0>)

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [11]:
net = FixedHiddenMLP()
net(X)

tensor(0.3637, grad_fn=<SumBackward0>)

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3391, grad_fn=<SumBackward0>)